<h1 align="center"><font color="yellow">DeepLearning.AI</font>: <font color="red">Overview of embeddings-based retrieval</font></h1>

<font color="yellow">Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

In [1]:
# from helper_utils import word_wrap
from pypdf import PdfReader

reader = PdfReader("spacy_teste.pdf") # "microsoft_annual_report_2022.pdf"
# https://s172-31-7-241p51602.lab-aws-production.deeplearning.ai/files/microsoft_annual_report_2022.pdf
pdf_texts = [p.extract_text().strip() for p in reader.pages]

# Filter the empty strings
pdf_texts = [text for text in pdf_texts if text]

pdf_texts


['CHATGPT: Granular ity clust ering\nQ: Que significa a granular idade de um clust ering\nA granularidade em um clustering refere-se ao nív el de detalhament o da divisão  \ndos dados em gr upos ou clust ers. \nPor exemplo, em um conjunto de dados de vendas de produtos em uma loja,  \nA escolha da granularidade depende do objetivo do clustering e da interpretação  \ndos resultados. Em alguns casos,  uma granular idade alta pode ajudar a  \nidentificar p adrões e caract erísticas específicas dos dados , enquanto em outros  \ncasos, uma granular idade b aixa pode ser suficient e para identificar t endências  \ngerais e insights.\nQ: Podes fazer ex emplos do conceit o\nClaro, aqui estão alguns exemplos para ilustrar o conceito de granularidade em  \nclustering:\nExemplo 1: Clust ering de compras online\nImagine que você esteja trabalhando em uma empresa de comércio eletrônico e  \nqueira agrupar os clientes com base em suas compras.  \nExemplo 2: Clust ering de dados de saúde\nSuponha que

In [3]:
len(pdf_texts)

5

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter


character_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", ". ", " ", ""],
                                                    chunk_size=400,
                                                    chunk_overlap=0
                                                   )

character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))

#print(word_wrap(character_split_texts[10]))
print(f"\nTotal chunks: {len(character_split_texts)}")



Total chunks: 30


In [5]:
token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)

token_split_texts = []
for text in character_split_texts:
    token_split_texts += token_splitter.split_text(text)

#print(word_wrap(token_split_texts[10]))
print(f"\nTotal chunks: {len(token_split_texts)}")


/home/eddygiusepe/1_Eddy_Giusepe/RAG_OpenAI_Weaviate_and_LangChain/venv_Weaviate/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
.gitattributes: 100%|██████████| 1.18k/1.18k [00:00<00:00, 10.6MB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 1.61MB/s]
README.md: 100%|██████████| 10.6k/10.6k [00:00<00:00, 12.3MB/s]
config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 276kB/s]
data_config.json: 100%|██████████| 39.3k/39.3k [00:00<00:00, 11.6MB/s]
pytorch_model.bin:  45%|████▌     | 199M/438M [00:46<00:37, 6.42MB/s] Error while downloading from https://cdn-lfs.huggingface.co/sentence-transformers/all-mpnet-base-v2/a8fd120b1a0032e70ff3d4b8ab8e46a6d01c2cb08ffe7c007a021c1788928146?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+file


Total chunks: 30


In [6]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

embedding_function = SentenceTransformerEmbeddingFunction()



print(embedding_function([token_split_texts[10]]))


[[0.10132455825805664, -0.0053729768842458725, 0.017298424616456032, -0.012371456250548363, 0.03420179709792137, -0.05840475857257843, 0.09047814458608627, 0.007839850150048733, 0.0032240378204733133, -0.01637033373117447, 0.11403467506170273, -0.08292246609926224, 0.09663739055395126, -0.007726727519184351, 0.05186151713132858, -0.002709528198465705, 0.018579838797450066, 0.03599350526928902, -0.018323931843042374, 0.028828157112002373, 0.027717048302292824, 0.013960310257971287, -0.10501343756914139, 0.06958600878715515, -0.08404848724603653, 0.0965137854218483, 0.025257686153054237, -0.052407097071409225, -0.003427807241678238, -0.028471950441598892, -0.05142757296562195, 0.03001488372683525, 0.15323688089847565, 0.032849136739969254, 0.043266113847494125, 0.04894221946597099, 0.07249372452497482, 0.04137548804283142, -0.06749024242162704, 0.07968004792928696, -0.04567169398069382, 0.025134440511465073, 0.005341050680726767, -0.09687402844429016, -0.03448357433080673, -0.08194494247

In [7]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("spacy_eddy", embedding_function=embedding_function)

ids = [str(i) for i in range(len(token_split_texts))]

chroma_collection.add(ids=ids, documents=token_split_texts)
chroma_collection.count()

30

In [8]:
query = "O que é entropia?"

results = chroma_collection.query(query_texts=[query], n_results=5)
retrieved_documents = results['documents'][0]

for document in retrieved_documents:
    #print(word_wrap(document))
    print(document)
    print('\n')

hipertensao " e " pacientes com doencas cardiacas ". se a granular idade do clust ering for b aixa, voce pode agrupar os usuarios em grandes categorias, como " usuarios que gostam de esportes " e " usuarios que gostam de musica ". se a granular idade do clust ering for alta, voce pode agrupar os usuarios em clusters especificos, como " usuarios que gostam de futebol ", " usuarios que


e significativas dos dados. 4.


coeficiente de variacao total. essas metricas podem ajuda - lo a determinar se o numero de clusters escolhido e apropriado para seus dados. 3. consider e o cont exto : finalmente, e importante considerar o contexto em que os dados estao sendo analisados. dependendo do objetivo da analise, a granularidade pode precisar ser ajustada para capturar informacoes relevantes


muit o distint os e nao ha sobr eposicao entr e eles. isso significa que e possiv el sep arar os dados de amb as as classes com alta pr ecisao. 3.


em geral, o objetivo do problema de classificacao e encon

In [9]:
# Substitua sua chave de API OpenAI:
import openai
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.environ['OPENAI_API_KEY']


openai_client = OpenAI()

In [10]:
def rag(query, retrieved_documents, model="gpt-3.5-turbo"):
    information = "\n\n".join(retrieved_documents)

    messages = [
        {
            "role": "system",
            "content": "Você é um assistente especializado e útil em pesquisa Granularidade e Clusterização. Seus usuários estão fazendo perguntas sobre as informações contidas em um documento de pesquisa."
            "Será mostrada a pergunta do usuário e as informações relevantes do documento de pesquisa. Responda à pergunta do usuário usando apenas essas informações."
        },
        {"role": "user", "content": f"Question: {query}. \n Information: {information}"}
    ]
    
    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content



In [11]:
output = rag(query=query, retrieved_documents=retrieved_documents)

#print(word_wrap(output))
print(output)


A entropia não é mencionada nas informações fornecidas sobre granularidade e clusterização. Não posso responder a esse questionamento com base nas informações fornecidas.
